In [8]:
import ee
import rasterio
import numpy as np
import geemap

In [9]:
ee.Initialize()

In [10]:
#Raster, die berechnet werden sollen aus Google Drive laden

S2_1 = ee.Image("projects/gis12025/assets/S2_1_Alison_Reef")
S2_2 = ee.Image("projects/gis12025/assets/S2_2_Alison_Reef")


In [11]:
#NDWI berechnen
#Wegen hinzufügen zu assets heißen die Bänder nicht mehr B3 und B8 sondern b1 und b2 aber entsprechen denen

ndwi_1 = S2_1.normalizedDifference(['b1','b2']).rename('NDWI') 
ndwi_2 = S2_2.normalizedDifference(['b1','b2']).rename('NDWI')

In [15]:
#Land = 1, Wasser = 0
#NDWI <=0 = Land

land_1 = ndwi_1.lte(0).unmask(0).rename('Land') #lte = less than or equal to, mit dem Wert kann man noch rumspielen
land_2 = ndwi_2.lte(0).unmask(0).rename('Land')


In [16]:
#Landgewinn = Pixel, die vorher Wasser waren und jetzt Land sind sprich land_2 = 1 AND land_1 = 0

land_gain = land_2.And(land_1.Not())

In [17]:
#Landgewinn exportieren

geemap.ee_export_image_to_drive(
    image=land_gain,               
    description='land_gain_Alison_Reef_unmask',       
    folder='Land_Gain_Exports',    
    scale=10,                      
    crs='EPSG:32649',              #sc Meer
)

In [18]:
#Werte printen

#Land-Gain-TIFF als Asset laden
results = ee.Image("projects/gis12025/assets/land_gain_Alison_Reef_unmask")

#Pixel-Fläche berechnen; wegen Koordinatensystem sind es m²
pixel_area = ee.Image.pixelArea()

#Landgewinn-Fläche berechnen
land_gain_area = results.multiply(pixel_area)

#Fläche aufsummieren
area_stats = land_gain_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=results.geometry(),  # gesamte Ausdehnung des TIFFs
    scale=10,
)

area_m2 = area_stats.getInfo()
print("Landgewinn in m²:", area_m2)

Landgewinn in m²: {'b1': 37427.79383087158}
